# Lab 4: Diputados

El objetivo de este lab es que pongas en práctica tus habilidades de recopilación de datos usando web scraping, así como tu capacidad de analizar dichos datos ya que la información se ecuentra estructurada.

## Instrucciones
Desarrolla un programa que recopile la información de cada disputado listado en la página de gobernación: http://sitl.diputados.gob.mx/LXIV_leg/listado_diputados_buscador.php y recopila la siguiente información:

* Nombre del diputado
* Entidad que representa
* Partido político
* Correo electrónico
* Nivel de escolaridad (secundaria, preparatoria, universidad, maestría, doctorado)
* Iniciativas presentadas y estatus ante el Pleno (Tipo de presentación, totales y aprobadas)

Una vez que tengas toda la información capturada en dataframe haz un análisis basado en las siguientes puntos (tu puedes proponer más):
* Cantidad de diputados por partido político
* Niveles de escolaridad
* Niveles de escolaridad por partido político
* ¿Cómo se ve la cantidad de propuestas realizadas y aprobadas con respecto a?:
    * Partido político
    * Escolaridad

Nota: Acompaña tus análisis con plots que nutran tu análisis

In [1]:
import pandas as pd
from urllib.request import urlopen
import bs4
from bs4 import BeautifulSoup
import numpy as np

In [2]:
def create_soup(url):
    source = urlopen(url).read()
    return BeautifulSoup(source)

In [3]:
gen_url = "http://sitl.diputados.gob.mx/LXIV_leg/listado_diputados_buscador.php"
gen_soup = create_soup(gen_url)
gen_soup



<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script language="JavaScript" src="./javascripts/anim_imag.js" type="text/javascript"></script>
<link href="./include/styles_diputados.css" rel="stylesheet" type="text/css"/>
<link href="./include/formatonvo.css" rel="stylesheet" type="text/css"/>
<title>Listado de Diputados LXIV</title>
</head>
<body leftmargin="0" topmargin="0">
<table align="center" border="0" cellpadding="0" cellspacing="0">
<tr>
<td>
<tr>
<td valign="top" width="760"><img border="0" height="115" src="images/cabezal.jpg" width="760"/></td>
</tr>
<tr>
<td colspan="3">
<table border="0" cellpadding="0" cellspacing="0" width="760">
<tr>
<td align="center" bgcolor="#595843" class="textoBlanco" width="175">31/03/2022</td>
<td align="right" width="60"><a href="http://www.diputados.gob.mx" onmouseout="MM_swapImgRestore()" onmouseover="MM

In [4]:
anchor_tags = gen_soup.find_all('a', attrs={'class': 'linkVerde'})[1:]
anchor_tags[0:10]

[<a class="linkVerde" href="curricula.php?dipt=484">1 Abasolo Sánchez Pedro Daniel</a>,
 <a class="linkVerde" href="curricula.php?dipt=476">2 Acosta Peña Brasil Alberto</a>,
 <a class="linkVerde" href="curricula.php?dipt=423">3 Adame Castillo Marco Antonio</a>,
 <a class="linkVerde" href="curricula.php?dipt=250">4 Aguilar Castillo Heriberto Marcelo</a>,
 <a class="linkVerde" href="curricula.php?dipt=126">5 Aguilar Castillo Ricardo</a>,
 <a class="linkVerde" href="curricula.php?dipt=289">6 Aguilar Linda Bonifacio</a>,
 <a class="linkVerde" href="curricula.php?dipt=57">7 Aguilar Molina Leticia Arlett</a>,
 <a class="linkVerde" href="curricula.php?dipt=345">8 Aguilar Vega Marcos</a>,
 <a class="linkVerde" href="curricula.php?dipt=333">9 Aguilera Brenes Sebastián</a>,
 <a class="linkVerde" href="curricula.php?dipt=73">10 Aguilera Chairez Maribel</a>]

In [5]:
links = ['http://sitl.diputados.gob.mx/LXIV_leg/'+ anchor_tag.get('href') for anchor_tag in anchor_tags]
links[0:10]

['http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=484',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=476',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=423',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=250',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=126',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=289',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=57',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=345',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=333',
 'http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=73']

In [6]:
# helper functions
def determine_party_from_img_src(src):
    if src == 'images/LogoMorena.jpg':
        return 'Morena'
    elif src == 'images/pan.png':
        return 'PAN'
    elif src == 'images/pri01.png':
        return 'PRI'
    elif src == 'images/Logo_PT.png':
        return 'PT'
    elif src == 'images/logo_movimiento_ciudadano.png':
        return 'MC'
    elif src == 'images/encuentro.jpg':
        return 'PES'
    elif src == 'images/prd01':
        return 'PRD'
    elif src == 'images/logvrd.jpg':
        return 'Partido Verde'
    else:
        return 'Sin Partido'

def find_keys_for_initiatives(soup):
    possible_keys = [td.contents[0] for td in soup.find_all('td', attrs={'class': 'textoNegro', 'width': 150})]
    first_occurrence_of_string = possible_keys.index('TOTAL')
    keys = possible_keys[first_occurrence_of_string+1:]
    keys1 = ['Propuestas_' + key for key in keys]
    keys2 = ['Propuestas aprobadas_' + key for key in keys]
    return [keys1, keys2]


In [7]:
# driver function to get personal data for congressperson
def get_personal_data(soup): 
    strong_tags = soup.find_all('strong')
    tds = soup.find('td', attrs={'class': 'textoNegro'})
    name = strong_tags[0].contents[0][5:]
    entity = strong_tags[2].contents[0]
    email = strong_tags[5].contents[0]
    studies = tds.contents[0] if len(tds) > 0 else 'Sin estudios registrados'
    party = determine_party_from_img_src(soup.find_all('img')[2].get('src'))
    return {
        'Nombre': name,
        'Entidad': entity,
        'Correo electrónico': email,
        'Partido': party,
        'Escolaridad': studies
    }


In [8]:
# driver function to get data on congressperon's initiatives
def get_initiatives_data(soup):
    initiative_dict = dict()
    keys_for_proposed_initiatives, keys_for_approved_initiatives = find_keys_for_initiatives(soup)
    tds = soup.find_all('table', attrs={'width': 775})[1].find_all('td')[3:-1]
    tds = [td.contents[0] for td in tds][7:]
    # return an empty dict if there are no initiatives
    if len(tds) == 0:
        for key in keys_for_proposed_initiatives:
            initiative_dict[key] = 0
        for key in keys_for_approved_initiatives:
            initiative_dict[key] = 0
        return initiative_dict
    tds_values = []
    for td in tds:
        if isinstance(td, bs4.Tag):
            tds_values.append(td.contents[0])
        else: 
            tds_values.append(td)
    initiative_array = np.array(tds_values)
    initiative_table = np.reshape(initiative_array, (len(keys_for_approved_initiatives),7))
    for i, row in enumerate(initiative_table):
        initiative_dict[keys_for_proposed_initiatives[i]] = int(row[1])
        initiative_dict[keys_for_approved_initiatives[i]] = int(row[2])
    return initiative_dict


In [9]:
# main function to read the HTML and return the congressperson's data as a dictionary
def read_one_soup(soup):
    personal_data_dict = get_personal_data(soup)
    initiatives_dict = get_initiatives_data(soup)
    total_data_dict = {**personal_data_dict, **initiatives_dict}
    return total_data_dict
    

In [217]:
congresspeople_data = []


for link in links:
    try:
        soup = create_soup(link)
        congressperson = read_one_soup(soup)
        congresspeople_data.append(congressperson)
    except Exception as e:
        print(f'Exception ocurred with link {link}')
        print(str(e))


In [218]:

congress_df = pd.DataFrame(congresspeople_data)

congress_df

,Nombre,Entidad,Correo electrónico,Partido,Escolaridad,Propuestas_Adherente,Propuestas aprobadas_Adherente,Propuestas_Diversos Grupos Parlamentarios,Propuestas aprobadas_Diversos Grupos Parlamentarios,Propuestas_De Grupo,Propuestas aprobadas_De Grupo,Propuestas_TOTAL,Propuestas aprobadas_TOTAL,Propuestas_Iniciante,Propuestas aprobadas_Iniciante,Propuestas_Proponente,Propuestas aprobadas_Proponente
0,Pedro Daniel Abasolo Sánchez,México,pedro.abasolo@diputados.gob.mx,PT,Secundaria,1.0,0.0,1.0,0.0,22.0,12.0,24,12,NaN,NaN,NaN,NaN
1,Brasil Alberto Acosta Peña,México,brasil.acosta@diputados.gob.mx,PRI,Doctorado,51.0,10.0,1.0,0.0,9.0,0.0,73,11,12.0,1.0,NaN,NaN
2,Marco Antonio Adame Castillo,Morelos,marco.adame@diputados.gob.mx,PAN,Maestría,112.0,15.0,5.0,1.0,37.0,4.0,164,21,10.0,1.0,NaN,NaN
3,Heriberto Marcelo Aguilar Castillo,Sonora,heriberto.aguilar@diputados.gob.mx,Morena,Licenciatura,10.0,1.0,2.0,0.0,29.0,14.0,53,16,12.0,1.0,NaN,NaN
4,Ricardo Aguilar Castillo,México,raguilar.castillo@diputados.gob.mx,PRI,Licenciatura,51.0,9.0,4.0,1.0,7.0,0.0,67,10,5.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Claudia Valeria Yáñez Centeno y Cabrera,Colima,claudia.yanezcenteno@diputados.gob.mx,Morena,Otro,29.0,7.0,1.0,0.0,27.0,14.0,60,21,3.0,0.0,NaN,NaN
496,Casimiro Zamora Valdez,Sinaloa,casimiro.zamora@diputados.gob.mx,Morena,Licenciatura,15.0,2.0,3.0,0.0,27.0,14.0,49,16,4.0,0.0,NaN,NaN
497,Martha Angélica Zamudio Macias,Jalisco,martha.zamudio@diputados.gob.mx,MC,Carrera Trunca,18.0,3.0,7.0,0.0,35.0,3.0,77,9,17.0,3.0,NaN,NaN
498,Graciela Zavaleta Sánchez,Oaxaca,graciela.zavaleta@diputados.gob.mx,Morena,Primaria,7.0,3.0,1.0,0.0,27.0,14.0,48,19,13.0,2.0,NaN,NaN


In [219]:
congress_df.to_csv('congress_dataset.csv')